<a href="https://colab.research.google.com/github/oumeimaghnimi/Huggingface_hub_learning/blob/master/Copie_de_The_push_to_hub_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook regroups the code sample of the video below, which is a part of the [Hugging Face course](https://huggingface.co/course).

In [1]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/A5IWIxsHLUw?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Install the Transformers and Datasets libraries to run this notebook.

In [4]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 264 kB 8.2 MB/s 
     |████████████████████████████████| 2.6 MB 58.9 MB/s 
     |████████████████████████████████| 118 kB 55.3 MB/s 
     |████████████████████████████████| 243 kB 70.6 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 3.3 MB 58.8 MB/s 
     |████████████████████████████████| 636 kB 59.9 MB/s 
     |████████████████████████████████| 895 kB 60.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


You will need an authentication token with your Hugging Face credentials to use the `push_to_hub` method. Execute `huggingface-cli login` in your terminal or by uncommenting the following cell:

In [5]:
#!huggingface-cli login
!transformers-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: oumeima
Password: 
Login successful
Your token: IdlHLMifuinaYGfKtmyBFlCKpztZaHklVWKqXdqRoEUskQkVjXntXYrAfMLLOusJwpovVMlBEDqTqnlEHwHlkdBbAZnyveUmAOAnIxSUOVQZqHiqjqRNMlsJJgyTmGBk 

Your token has been saved to /root/.huggingface/token


In [6]:
import numpy as np

from datasets import load_dataset, load_metric
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

In [7]:
checkpoint = "bert-base-cased"

In [8]:
raw_datasets = load_dataset("glue", "mrpc")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments(
    "finetuned-bert-mrpc",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    log_level="error",
    push_to_hub=True,
    push_to_hub_model_id="finetuned-bert-mrpc",
    # push_to_hub_organization="huggingface",
     push_to_hub_token="IdlHLMifuinaYGfKtmyBFlCKpztZaHklVWKqXdqRoEUskQkVjXntXYrAfMLLOusJwpovVMlBEDqTqnlEHwHlkdBbAZnyveUmAOAnIxSUOVQZqHiqjqRNMlsJJgyTmGBk",
)

data_collator = DataCollatorWithPadding(tokenizer)

metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [9]:
!python -m pip install --upgrade pip; pip install git-lfs
#git lfs install
!apt-get install git-lfs


     |████████████████████████████████| 1.6 MB 7.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,691 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting

In [10]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.570400,0.420420,0.791667,0.854202
2,0.339100,0.415668,0.845588,0.895522
3,0.192300,0.528038,0.852941,0.900332


TrainOutput(global_step=690, training_loss=0.36729109390922215, metrics={'train_runtime': 180.7776, 'train_samples_per_second': 60.87, 'train_steps_per_second': 3.817, 'total_flos': 445479905606400.0, 'train_loss': 0.36729109390922215, 'epoch': 3.0})

## Push to hub from the Trainer directly

The `Trainer` has a new method to directly upload the model, tokenizer and model configuration in a repo on the [Hub](https://huggingface.co/). It will even auto-generate a model card draft using the hyperparameters and evaluation results!

In [14]:

!git config --global user.email "oumeimaghnimi29@gmail.com"
!git config --global user.name "oumeima"
trainer.push_to_hub()

'https://huggingface.co/oumeima/finetuned-bert-mrpc/commit/63d0302ab1d8709b53bf0e216b486d95647ede77'

If you are using your own training loop, you can push the model and tokenizer separately (and you will have to write the model card yourself):

In [ ]:
# model.push_to_hub("finetuned-bert-mrpc")
# tokenizer.push_to_hub("finetuned-bert-mrpc")

## You can load your model from anywhere using from_pretrained!

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "sgugger/finetuned-bert-mrpc"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

## You can use your model in a pipeline!

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model_name)

In [ ]:
classifier("My name is Sylvain. [SEP] My name is Lysandre")

## Updating a problematic file is super easy!

In [ ]:
model.config.label2id = {"not equivalent": 0, "equivalent": 1}

In [ ]:
model.config.id2label = {0: "not equivalent", 1: "equivalent"}

In [ ]:
model.config.push_to_hub("finetuned-bert-mrpc")

In [ ]:
classifier = pipeline("text-classification", model=model_name)

classifier("My name is Sylvain. [SEP] My name is Lysandre")